In [1]:
import numpy as np
import pandas as pd
import xarray as xr
from joblib import dump, load
from sklearn.model_selection import train_test_split
import math as mt
from math import e
from sklearn.preprocessing import StandardScaler

In [2]:
# Read Orography
OroData = xr.open_dataset('../../../Data/eraDown/ERA5_2degree_Down/DailyMean/ERA5IGP_Orography.nc')

In [3]:
#Surface data
t2mData = xr.open_dataset('../../../Data/CMIP6/MPI-ESM1-2-LR/processed/Regrid_tas_day_MPI-ESM1-2-LR_historical_r1i1p1f1_gn_18500101-20141231.nc')
rhData  = xr.open_dataset('../../../Data/CMIP6/MPI-ESM1-2-LR/processed/Regrid_hurs_day_MPI-ESM1-2-LR_historical_r1i1p1f1_gn_18500101-20141231.nc')
u10Data = xr.open_dataset('../../../Data/CMIP6/MPI-ESM1-2-LR/processed/Regrid_uas_day_MPI-ESM1-2-LR_historical_r1i1p1f1_gn_18500101-20141231.nc')
v10Data = xr.open_dataset('../../../Data/CMIP6/MPI-ESM1-2-LR/processed/Regrid_vas_day_MPI-ESM1-2-LR_historical_r1i1p1f1_gn_18500101-20141231.nc')

In [4]:
# Level Data
tLevData = xr.open_dataset('../../../Data/CMIP6/MPI-ESM1-2-LR/processed/Regrid_ta_day_MPI-ESM1-2-LR_historical_r1i1p1f1_gn_18500101-20141231.nc')
zLevData = xr.open_dataset('../../../Data/CMIP6/MPI-ESM1-2-LR/processed/Regrid_zg_day_MPI-ESM1-2-LR_historical_r1i1p1f1_gn_18500101-20141231.nc')
wLevData = xr.open_dataset('../../../Data/CMIP6/MPI-ESM1-2-LR/processed/Regrid_wap_day_MPI-ESM1-2-LR_historical_r1i1p1f1_gn_18500101-20141231.nc')
uLevData = xr.open_dataset('../../../Data/CMIP6/MPI-ESM1-2-LR/processed/Regrid_ua_day_MPI-ESM1-2-LR_historical_r1i1p1f1_gn_18500101-20141231.nc')

In [5]:
t2mD = t2mData.sel(latitude=slice(32,24),longitude=slice(74,86))
u10D = u10Data.sel(latitude=slice(32,24),longitude=slice(74,86))
v10D = v10Data.sel(latitude=slice(32,24),longitude=slice(74,86))
rhD  = rhData.sel(latitude=slice(32,24),longitude=slice(74,86))
tLevD = tLevData.sel(latitude=slice(32,24),longitude=slice(74,86))
uLevD = uLevData.sel(latitude=slice(32,24),longitude=slice(74,86))
w700D = wLevData.sel(latitude=slice(32,24),longitude=slice(74,86)).sel(plev=70000,method='nearest')
zLevD = zLevData.sel(latitude=slice(32,24),longitude=slice(74,86))
OroD = OroData.sel(latitude=slice(32,24),longitude=slice(74,86))
#w700D

In [6]:
# Calculate wind speed and relative humidity inv  ushear
ws = ((v10D.vas.values**2)+(u10D.uas.values**2))**0.5
ws_ds = xr.Dataset({'ws': (('time','latitude','longitude'), ws)},
                   coords={'time': v10D.time,'latitude': v10D.latitude,'longitude': v10D.longitude})

rh_ds = xr.Dataset({'rh': (('time','latitude','longitude'), rhD.hurs)},
                   coords={'time': v10D.time,'latitude': v10D.latitude,'longitude': v10D.longitude})

#Calculate inv
inv=t2mD.tas.values-tLevD.ta.sel(plev=85000,method='nearest').values
inv_ds = xr.Dataset({'inv': (('time','latitude','longitude'), inv)}, 
                    coords={'time': v10D.time,'latitude': v10D.latitude,'longitude': v10D.longitude})
inv_ds.attrs
inv_ds.attrs['units']='K'
inv_ds.attrs['long_name']='t2m - t850'

#u shear calculation
ushear=(uLevD.ua.sel(plev=85000,method='nearest').values-u10D.uas.values)/(zLevD.zg.sel(plev=85000,method='nearest').values) 
ushear_ds = xr.Dataset({'ushear': (('time','latitude','longitude'), ushear)}, 
                       coords={'time': v10D.time,'latitude': v10D.latitude,'longitude': v10D.longitude})
ushear_ds.attrs['units']='s-1'
ushear_ds.attrs['long_name']='(u10 - u850)/z850'

In [7]:
AOData = xr.open_dataset('MPI-ESM1-2-LR-AOindex-NDJF-Daily-1980-2014.nc')

AO=AOData.AO
#datetimeindex = AO.indexes['time'].to_datetimeindex()#
#datetimeindex
#AO['time'] = datetimeindex
AO.values

AO5D=AO.rolling(time=5).mean()

AO5DAll=AO5D[((AO5D.time.dt.month>11) | (AO5D.time.dt.month<2)) & 
             (AO5D.time.dt.year<2020)].sel(time=slice('1980-1-1','2018-12-31'))
#ao

In [8]:
EUData = xr.open_dataset('MPI-ESM1-2-LR-EUindex-NDJF-Daily-1980-2014.nc')
EU=EUData.EU
#datetimeindex = EU.indexes['time'].to_datetimeindex()
#datetimeindex
#EU['time'] = datetimeindex
EU.values
EU5D=EU.rolling(time=5).mean()

EU5DAll=EU5D[((EU5D.time.dt.month>11) | (EU5D.time.dt.month<2)) & 
             (EU5D.time.dt.year<2020)].sel(time=slice('1980-1-1','2018-12-31'))


In [9]:
# create mask
oro = OroD.z
oro.values = OroD.z.values/9.81
oro.attrs
oro.attrs['units']='meter'
oro.attrs['long_name']='Orography'
oro.values[oro.values>500.1]=np.NaN
mask=oro.values/oro.values

In [10]:
# Apply Mask 
t2mD.tas.values=t2mD.tas.values*mask
ws_ds.ws.values=ws_ds.ws.values*mask
rh_ds.rh.values=rh_ds.rh.values*mask
w700D.wap.values=w700D.wap.values*mask
inv_ds.inv.values=inv_ds.inv.values*mask
ushear_ds.ushear.values=ushear_ds.ushear.values*mask

In [11]:
# Feild mean
t2mTS1=t2mD.mean(dim=['latitude','longitude'], skipna=True).tas
wsTS1=ws_ds.mean(dim=['latitude','longitude'], skipna=True).ws
rhTS1=rh_ds.mean(dim=['latitude','longitude'], skipna=True).rh
wTS1=w700D.mean(dim=['latitude','longitude'], skipna=True).wap
invTS1=inv_ds.mean(dim=['latitude','longitude'], skipna=True).inv
ushearTS1=ushear_ds.mean(dim=['latitude','longitude'], skipna=True).ushear

In [12]:
# shift one day so we can take values of previous day
t2mTS=t2mTS1.shift(time=1)
wsTS=wsTS1.shift(time=1)
rhTS=rhTS1.shift(time=1)
wTS=wTS1.shift(time=1)
invTS=invTS1.shift(time=1)
ushearTS=ushearTS1.shift(time=1)

In [13]:
# Select required time 
t2m=t2mTS[((t2mTS.time.dt.month>11) | (t2mTS.time.dt.month<2)) & (t2mTS.time.dt.year<2020)].sel(time=slice('1980-1-1','2014-12-31'))
ws=wsTS[((wsTS.time.dt.month>11) | (wsTS.time.dt.month<2)) & (wsTS.time.dt.year<2020)].sel(time=slice('1980-1-1','2014-12-31'))
inv=invTS[((wsTS.time.dt.month>11) | (wsTS.time.dt.month<2)) & (wsTS.time.dt.year<2020)].sel(time=slice('1980-1-1','2014-12-31'))
rh=rhTS[((wsTS.time.dt.month>11) | (wsTS.time.dt.month<2)) & (wsTS.time.dt.year<2020)].sel(time=slice('1980-1-1','2014-12-31'))
w=wTS[((wsTS.time.dt.month>11) | (wsTS.time.dt.month<2)) & (wsTS.time.dt.year<2020)].sel(time=slice('1980-1-1','2014-12-31'))
ushear=ushearTS[((wsTS.time.dt.month>11) | (wsTS.time.dt.month<2)) & (wsTS.time.dt.year<2020)].sel(time=slice('1980-1-1','2014-12-31'))

In [14]:
X       = pd.DataFrame([t2m.values,ws.values,rh.values/100.0,inv.values,w.values,ushear.values*100.0,AO5DAll.values,
                        EU5DAll.values],
                       index=['t2m', 'ws', 'rh','inv', 'w', 'ushear', 'AO5D', 'EU5D']).T[:]
X.index = pd.to_datetime(t2m.time.values)
X.describe(include='all')

,t2m,ws,rh,inv,w,ushear,AO5D,EU5D
count,2170.000000,2170.000000,2170.000000,2170.000000,2170.000000,2170.000000,2170.000000,2170.000000
mean,286.986919,1.778659,0.463905,3.199970,0.016084,0.032836,-0.063573,-0.033758
std,2.156770,0.504330,0.124671,1.923724,0.103717,0.115755,0.930857,0.592370
min,279.506927,0.604827,0.178244,-3.213243,-0.447715,-0.351528,-2.504789,-1.892424
25%,285.676270,1.429569,0.371191,1.815930,-0.039987,-0.046358,-0.747668,-0.469864
50%,287.096710,1.693228,0.448470,3.311842,0.030168,0.032021,-0.046287,-0.016551
75%,288.488281,2.027122,0.538146,4.666699,0.084373,0.104276,0.534930,0.405586
max,292.566833,4.535131,0.902833,7.286118,0.367549,0.406510,2.843463,1.685801


In [15]:
norm = StandardScaler().fit(X)
    # transform training data
X = norm.transform(X)
X
X = pd.DataFrame(norm.transform(X))
X.index = pd.to_datetime(t2m.time.values)
X.describe(include='all')

,0,1,2,3,4,5,6,7
count,2170.000000,2170.000000,2170.000000,2170.000000,2170.000000,2170.000000,2170.000000,2170.000000
mean,-133.093968,-3.527585,-3.721895,-1.663808,-0.155109,-0.283729,0.068311,0.057002
std,0.463870,1.983741,8.024820,0.520065,9.646087,8.642882,1.074774,1.688912
min,-134.702736,-8.144752,-22.109337,-3.397574,-43.290318,-28.982263,-2.750332,-5.242257
25%,-133.375857,-4.900702,-9.689713,-2.037973,-5.369955,-6.196689,-0.721549,-1.186383
50%,-133.070354,-3.863618,-4.715383,-1.633564,1.154803,-0.344584,0.088270,0.106061
75%,-132.771060,-2.550276,1.056849,-1.267289,6.196120,5.050375,0.759347,1.309619
max,-131.893860,7.314762,24.531026,-0.559148,32.532571,27.616627,3.424794,4.959653


In [16]:
regLin=load('../../March2021/Observation_models/LRModel.joblib') 

In [17]:
yLR=regLin.predict(X)
y_predLin_ds=xr.Dataset({'yLR': (('time'), yLR)}, coords={'time': t2m.time})

In [18]:
dump(y_predLin_ds.yLR,'../Model_plots/MPI-ESM1-2-LR-LR-Y.joblib')

['../Model_plots/MPI-ESM1-2-LR-LR-Y.joblib']

In [19]:
regLin2=load('../../March2021/Observation_models/SVRModel.joblib') 
yLR=regLin2.predict(X)
y_predLin_ds=xr.Dataset({'yLR': (('time'), yLR)}, coords={'time': t2m.time})
dump(y_predLin_ds.yLR,'../Model_plots/MPI-ESM1-2-LR-SVR-Y.joblib')

['../Model_plots/MPI-ESM1-2-LR-SVR-Y.joblib']